In [1]:
import gzip
from pathlib import Path
from io import BufferedReader, BytesIO
from teleparser.decoders.ber import BerDecoder
from teleparser.file_handling import BufferManager

folder = Path.cwd().parent / "data"

In [2]:
file = folder / "timvoz.gz"
buffer_manager = BufferManager(file)

In [3]:
ber = BerDecoder()
file_buffer = buffer_manager.open_file()


In [5]:
if (tlv := ber.decode_tlv(file_buffer)) is not None:
    print("Varredura: (Call Data Record)")
    print(tlv.tag.string, tlv.tag.tag_class, tlv.tag.number, tlv.length, tlv.value)
    if tlv.children is not None:
        for child in tlv.children:
            print(80*'=')
            print("Call Module")
            print(f"string={child.tag.string}, tipo={child.tag.number}, comprimento={child.length}, bytes={child.value}")
            print(80 * "=")
            print(f'Quantidade de parâmetros dentro do Call Module: {len(child.children)}')
            print("Parâmetros:")
            for c in child.children:
                print(
                    f"string={c.tag.string}, tipo={c.tag.number}, comprimento={c.length}, bytes={c.value}"
                )


Varredura: (Call Data Record)
a0 BerClass.CONTEXT 0 182 b'\xa0\x81\xb3\x9a\x01\x01\x8b\x04\x00\x00\x00\x00\x88\x03\x18\x0c\x0e\x93\x0fZCTA09P BR00308\x8c\x03\x00\x00\x00\x82\x03\x16\x08[\x90\x02\x00\x01\x89\x03\x0b\x00\x0f\x8a\x03\x0b\x00\x0f\x9f-\x02%?\x9fZ\x0cA\n#\x01\x00\x004\x89\x18\x11\x87\xf0\x9c\x02\x02\x17\x9f)\x02\x0e\xaa\x87\x01\x02\x83\x01\x01\x9f2\x08\x11U4\x89\x18\x11\x87\xf0\x80\x03\x00\x02\x01\x8d\x03\x00\x00\x00\x9f*\x01\x00\x9f\x1f\x01\x00\x98\x01\x02\x8f\x01\x00\x9f.\x05-\xa0 %?\x95\x06HLR437\x96\x0741TWYMI\x8e\x01\x00\x84\x07A4)\x00R!\xf0\x85\x07A@\x93\x88\x11q\x08\x9b\x016\x81\x03|;\xac'
Call Module
string=a0, tipo=0, comprimento=179, bytes=b'\x9a\x01\x01\x8b\x04\x00\x00\x00\x00\x88\x03\x18\x0c\x0e\x93\x0fZCTA09P BR00308\x8c\x03\x00\x00\x00\x82\x03\x16\x08[\x90\x02\x00\x01\x89\x03\x0b\x00\x0f\x8a\x03\x0b\x00\x0f\x9f-\x02%?\x9fZ\x0cA\n#\x01\x00\x004\x89\x18\x11\x87\xf0\x9c\x02\x02\x17\x9f)\x02\x0e\xaa\x87\x01\x02\x83\x01\x01\x9f2\x08\x11U4\x89\x18\x11\x87\xf0\x80\x03

In [ ]:
%%time
ber = BerDecoder()
with buffer_manager.open() as file_buffer:
    counter = 0
    while (tlv := ber.decode_tlv(file_buffer)) is not None:
        counter += 1

print(f"Numbers of CDR blocks: {counter}")